In [8]:
!pip install -r requirements.txt

In [59]:
import pandas as pd
from pandas import read_csv
from pathlib import Path
import sqlite3
import os

SQLITE_DB_FILE = 'my_data.db'

if os.path.exists(SQLITE_DB_FILE):
  os.remove(SQLITE_DB_FILE)

Path(SQLITE_DB_FILE).touch()
conn = sqlite3.connect('my_data.db')
c = conn.cursor()

# load every csv table contents
tabla_de_aerolineas = read_csv("tabla_de_aerolineas.csv")
tabla_de_aeropuertos = read_csv("tabla_de_aeropuertos.csv")
tabla_de_movimientos = read_csv("tabla_de_movimientos.csv")
tabla_de_vuelos = read_csv("tabla_de_vuelos.csv")

# generate every sqlite table
c.execute('''CREATE TABLE aerolineas (ID_AEROLINEA int, NOMBRE_AEROLINEA text)''')
c.execute('''CREATE TABLE aeropuertos (ID_AEROPUERTO int, NOMBRE_AEROPUERTO text)''')
c.execute('''CREATE TABLE movimientos (ID_MOVIMIENTO int, DESCRIPCION text)''')
c.execute('''CREATE TABLE vuelos (ID_AEROLINEA int, ID_AEROPUERTO int, ID_MOVIMIENTO int, DIA text)''')

# write the data to every sqlite table
tabla_de_aerolineas.to_sql('aerolineas', conn, if_exists='append', index = False)
tabla_de_aeropuertos.to_sql('aeropuertos', conn, if_exists='append', index = False)
tabla_de_movimientos.to_sql('movimientos', conn, if_exists='append', index = False)
tabla_de_vuelos.to_sql('vuelos', conn, if_exists='append', index = False)

print('success !')

success !


In [49]:
tabla_de_aerolineas

,ID_AEROLINEA,NOMBRE_AEROLINEA
0,1,Volaris
1,2,Aeromar
2,3,Interjet
3,4,Aeromexico


In [50]:
tabla_de_aeropuertos

,ID_AEROPUERTO,NOMBRE_AEROPUERTO
0,1,Benito Juarez
1,2,Guanajuato
2,3,La paz
3,4,Oaxaca


In [51]:
tabla_de_movimientos

,ID_MOVIMIENTO,DESCRIPCION
0,1,Salida
1,2,Llegada


In [52]:
tabla_de_vuelos

,ID_AEROLINEA,ID_AEROPUERTO,ID_MOVIMIENTO,DIA
0,1,1,1,2021-05-02
1,2,1,1,2021-05-02
2,3,2,2,2021-05-02
3,4,3,2,2021-05-02
4,1,3,2,2021-05-02
5,2,1,1,2021-05-02
6,2,3,1,2021-05-04
7,3,4,1,2021-05-04
8,3,4,1,2021-05-04


# Se requiere saber lo siguiente
1. ¿Cuál es el nombre aeropuerto que ha tenido mayor movimiento durante el año?
2. ¿Cuál es el nombre aerolínea que ha realizado mayor número de vuelos durante
el año?
3. ¿En qué día se han tenido mayor número de vuelos?
4. ¿Cuáles son las aerolíneas que tienen más de 2 vuelos por día?

# 1. ¿Cuál es el nombre aeropuerto que ha tenido mayor movimiento durante el año?

In [85]:
pd.read_sql('''
SELECT count(*) as MOVIMIENTOS, NOMBRE_AEROPUERTO FROM vuelos v LEFT JOIN aeropuertos a ON v.ID_AEROPUERTO = a.ID_AEROPUERTO GROUP BY NOMBRE_AEROPUERTO
''', conn)

,MOVIMIENTOS,NOMBRE_AEROPUERTO
0,3,Benito Juarez
1,1,Guanajuato
2,3,La paz
3,2,Oaxaca


In [61]:
# Respuesta
pd.read_sql('''
SELECT MAX(MOVIMIENTOS) as MOVIMIENTOS, NOMBRE_AEROPUERTO from

(SELECT count(*) as MOVIMIENTOS, NOMBRE_AEROPUERTO FROM vuelos v LEFT JOIN aeropuertos a ON v.ID_AEROPUERTO = a.ID_AEROPUERTO GROUP BY NOMBRE_AEROPUERTO)

''', conn).to_dict('records')[0]

{'MOVIMIENTOS': 3, 'NOMBRE_AEROPUERTO': 'Benito Juarez'}

# 2. ¿Cuál es el nombre aerolínea que ha realizado mayor número de vuelos durante el año?

In [62]:
pd.read_sql('''

SELECT count(*) as VUELOS, NOMBRE_AEROLINEA FROM vuelos v LEFT JOIN aerolineas a ON v.ID_AEROLINEA = a.ID_AEROLINEA GROUP BY NOMBRE_AEROLINEA

''', conn)

,VUELOS,NOMBRE_AEROLINEA
0,3,Aeromar
1,1,Aeromexico
2,3,Interjet
3,2,Volaris


In [66]:
# Respuesta
pd.read_sql('''
SELECT MAX(VUELOS) as VUELOS, NOMBRE_AEROLINEA from

(SELECT count(*) as VUELOS, NOMBRE_AEROLINEA FROM vuelos v LEFT JOIN aerolineas a ON v.ID_AEROLINEA = a.ID_AEROLINEA GROUP BY NOMBRE_AEROLINEA)

''', conn).to_dict('records')[0]

{'VUELOS': 3, 'NOMBRE_AEROLINEA': 'Aeromar'}

# 3. ¿En qué día se han tenido mayor número de vuelos?

In [77]:
# Respuesta
pd.read_sql('''
SELECT count(DIA) as VUELOS, DIA FROM vuelos GROUP BY DIA
''', conn)

,VUELOS,DIA
0,6,2021-05-02
1,3,2021-05-04


# 4. ¿Cuáles son las aerolíneas que tienen más de 2 vuelos por día?

In [79]:
# (estoy reusando el código de la pregunta 2)
pd.read_sql('''
SELECT count(*) as VUELOS, NOMBRE_AEROLINEA FROM vuelos v LEFT JOIN aerolineas a ON v.ID_AEROLINEA = a.ID_AEROLINEA GROUP BY NOMBRE_AEROLINEA
''', conn)

,VUELOS,NOMBRE_AEROLINEA
0,3,Aeromar
1,1,Aeromexico
2,3,Interjet
3,2,Volaris


In [84]:
# Respuesta
pd.read_sql('''
SELECT VUELOS, NOMBRE_AEROLINEA from

(SELECT count(*) as VUELOS, NOMBRE_AEROLINEA FROM vuelos v LEFT JOIN aerolineas a ON v.ID_AEROLINEA = a.ID_AEROLINEA GROUP BY NOMBRE_AEROLINEA)

WHERE VUELOS > 2
''', conn)

,VUELOS,NOMBRE_AEROLINEA
0,3,Aeromar
1,3,Interjet
